<a href="https://colab.research.google.com/github/joshuaghannan/ECEC247_Project/blob/jonathan-tests/final_notebook_all_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### loading modules and data

In [1]:
########################################################

# If running with Google Colab

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [11]:
# # If using colab
 from google.colab import files
 files.upload()
# # select the 3 .py files (models, utils, data_utils)

{}

In [2]:
########################################################

# If running with Google Colab
# Create a folder "C247" and then store the project datasets within that folder
# Check that your datasets are setup correctly

!ls "/content/gdrive/My Drive/C247" # File path

data_utils.py	   person_test.npy	   X_test.npy	      y_train_valid.npy
EEG_loading.ipynb  person_train_valid.npy  X_train_valid.npy
models.py	   utils.py		   y_test.npy


In [1]:
from models import *
from utils import *
from data_utils import *

GPU is available


In [2]:
# X_test = np.load("X_test.npy")
# y_test = np.load("y_test.npy")
# person_train_valid = np.load("person_train_valid.npy")
# X_train_valid = np.load("X_train_valid.npy")
# y_train_valid = np.load("y_train_valid.npy")
# person_test = np.load("person_test.npy")

# Change if your directory is different

# dataset_path = './data/' # Yiming Path
dataset_path = "/content/gdrive/My Drive/C247/" 

X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

#train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### K-Fold Training and Validation (use k=1 to get results faster)

Data $\rightarrow$ Window $\rightarrow$ STFT

In [4]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 100
window_stride = 40
vote_num = 30 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 8
cwt_scale = 2.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 5

aug_1 = "cwt"
aug_2 = "window"


#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Data normalized...
Data shape: (2115, 22, 1000)
cwt applied...
fold 1
Initial fold shape: (1692, 176, 1000)
window applied...
Data shape: (37224, 176, 100)
RNN TYPE: GRU
WEIGHT DECAY: 0.001
LEARNING RATE: 0.001
Data prepared, model initialized, beginning training...
epoch: 1      time: 6.98    loss: 403.978    train acc: 0.270    val acc: 0.284
saving best model...
epoch: 2      time: 7.11    loss: 401.754    train acc: 0.284    val acc: 0.265
epoch: 3      time: 7.31    loss: 400.657    train acc: 0.294    val acc: 0.307
saving best model...
epoch: 4      time: 7.06    loss: 399.948    train acc: 0.296    val acc: 0.310
saving best model...
epoch: 5      time: 6.92    loss: 399.064    train acc: 0.304    val acc: 0.340
saving best model...
epoch: 6      time: 7.12    loss: 398.406    train acc: 0.305    val acc: 0.317
epoch: 7      time: 7.02    loss: 397.001    train acc: 0.317    val acc: 0.298
epoch: 8      time: 7.17    loss: 395.392    train acc: 0.327    val acc: 0.298
epoch: 9 

### Training and Testing

#### Test 1

In [9]:
print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.3160


In [3]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 150
window_stride = 50
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 8
cwt_scale = 2.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "cwt"
aug_2 = "window"


#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Data normalized...
Data shape: (2115, 22, 1000)
cwt applied...
fold 1
Initial fold shape: (1692, 176, 1000)
window applied...
Data shape: (28764, 176, 150)
RNN TYPE: GRU
WEIGHT DECAY: 0.001
LEARNING RATE: 0.001
Data prepared, model initialized, beginning training...
epoch: 1      time: 8.24    loss: 312.516    train acc: 0.269    val acc: 0.279
saving best model...
epoch: 2      time: 8.50    loss: 310.560    train acc: 0.285    val acc: 0.270
epoch: 3      time: 8.49    loss: 309.938    train acc: 0.292    val acc: 0.274
epoch: 4      time: 8.29    loss: 309.587    train acc: 0.296    val acc: 0.274
epoch: 5      time: 8.45    loss: 309.056    train acc: 0.304    val acc: 0.293
saving best model...
epoch: 6      time: 8.40    loss: 308.114    train acc: 0.308    val acc: 0.331
saving best model...
epoch: 7      time: 8.29    loss: 307.502    train acc: 0.312    val acc: 0.312
epoch: 8      time: 8.34    loss: 306.295    train acc: 0.322    val acc: 0.279
epoch: 9      time: 8.23    lo

In [4]:
print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Augmenting test data...
Testing...
Testing Accuracy: 0.3296


### Test 2

In [3]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 150
window_stride = 50
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 8
cwt_scale = 1.0

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 25
hidden_size = 50
lr = 1e-3
weight_decay = 2e-3
num_folds = 1

aug_1 = "cwt"
aug_2 = "window"


#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Data normalized...
Data shape: (2115, 22, 1000)
cwt applied...
fold 1
Initial fold shape: (1692, 176, 1000)
window applied...
Data shape: (28764, 176, 150)
RNN TYPE: GRU
WEIGHT DECAY: 0.002
LEARNING RATE: 0.001
Data prepared, model initialized, beginning training...
epoch: 1      time: 8.37    loss: 312.700    train acc: 0.263    val acc: 0.279
saving best model...
epoch: 2      time: 8.31    loss: 311.472    train acc: 0.272    val acc: 0.300
saving best model...
epoch: 3      time: 8.34    loss: 310.940    train acc: 0.278    val acc: 0.322
saving best model...
epoch: 4      time: 8.50    loss: 310.631    train acc: 0.281    val acc: 0.303
epoch: 5      time: 8.58    loss: 310.451    train acc: 0.284    val acc: 0.352
saving best model...
epoch: 6      time: 8.56    loss: 310.387    train acc: 0.281    val acc: 0.324
epoch: 7      time: 8.41    loss: 310.344    train acc: 0.284    val acc: 0.331
epoch: 8      time: 8.54    loss: 309.993    train acc: 0.286    val acc: 0.329
epoch: 9 

In [4]:
print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Augmenting test data...
Testing...
Testing Accuracy: 0.4424


### Test 3

In [4]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 200
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 9
cwt_scale = 1.0

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 30
hidden_size = 50
lr = 2e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "cwt"
aug_2 = "window"


#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Data normalized...
Data shape: (2115, 22, 1000)
cwt applied...
fold 1
Initial fold shape: (1692, 198, 1000)
window applied...
Data shape: (16920, 198, 200)
RNN TYPE: GRU
WEIGHT DECAY: 0.001
LEARNING RATE: 0.002
Data prepared, model initialized, beginning training...
epoch: 1      time: 7.30    loss: 185.216    train acc: 0.254    val acc: 0.274
saving best model...
epoch: 2      time: 7.06    loss: 184.458    train acc: 0.268    val acc: 0.227
epoch: 3      time: 7.10    loss: 184.099    train acc: 0.272    val acc: 0.281
saving best model...
epoch: 4      time: 7.15    loss: 183.935    train acc: 0.276    val acc: 0.279
epoch: 5      time: 7.14    loss: 183.745    train acc: 0.276    val acc: 0.291
saving best model...
epoch: 6      time: 7.17    loss: 183.832    train acc: 0.281    val acc: 0.253
epoch: 7      time: 7.24    loss: 183.434    train acc: 0.287    val acc: 0.298
saving best model...
epoch: 8      time: 7.27    loss: 183.627    train acc: 0.284    val acc: 0.324
saving be

In [5]:
print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Augmenting test data...
Testing...
Testing Accuracy: 0.3950


### Test 4

In [3]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 200
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 9
cwt_scale = 1.0

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 30
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "window"
aug_2 = "cwt"


#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Data normalized...
Data shape: (2115, 22, 1000)
window applied...
fold 1
Initial fold shape: (16920, 22, 200)
cwt applied...
Data shape: (16920, 198, 200)
RNN TYPE: GRU
WEIGHT DECAY: 0.001
LEARNING RATE: 0.001
Data prepared, model initialized, beginning training...
epoch: 1      time: 5.93    loss: 185.141    train acc: 0.258    val acc: 0.261
saving best model...
epoch: 2      time: 5.77    loss: 184.140    train acc: 0.273    val acc: 0.258
epoch: 3      time: 5.83    loss: 183.799    train acc: 0.280    val acc: 0.270
saving best model...
epoch: 4      time: 5.80    loss: 183.754    train acc: 0.280    val acc: 0.278
saving best model...
epoch: 5      time: 5.81    loss: 183.471    train acc: 0.288    val acc: 0.273
epoch: 6      time: 5.80    loss: 183.266    train acc: 0.286    val acc: 0.270
epoch: 7      time: 5.78    loss: 183.031    train acc: 0.294    val acc: 0.264
epoch: 8      time: 5.78    loss: 183.284    train acc: 0.286    val acc: 0.273
epoch: 9      time: 5.75    los

In [10]:
print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)

Augmenting test data...
Testing...


TypeError: ignored

### Test

In [3]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 9
cwt_scale = 0.7

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 30
hidden_size = 50
lr = 2e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "cwt"
aug_2 = "window"


#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Data normalized...
Data shape: (2115, 22, 1000)
cwt applied...
fold 1
Initial fold shape: (1692, 198, 1000)
window applied...
Data shape: (16920, 198, 250)
RNN TYPE: GRU
WEIGHT DECAY: 0.001
LEARNING RATE: 0.002
Data prepared, model initialized, beginning training...
epoch: 1      time: 8.69    loss: 185.440    train acc: 0.253    val acc: 0.284
saving best model...
epoch: 2      time: 8.81    loss: 184.310    train acc: 0.266    val acc: 0.258
epoch: 3      time: 8.79    loss: 184.026    train acc: 0.273    val acc: 0.322
saving best model...
epoch: 4      time: 8.83    loss: 183.658    train acc: 0.279    val acc: 0.348
saving best model...
epoch: 5      time: 8.70    loss: 183.818    train acc: 0.274    val acc: 0.248
epoch: 6      time: 8.73    loss: 183.609    train acc: 0.277    val acc: 0.270
epoch: 7      time: 8.65    loss: 183.554    train acc: 0.278    val acc: 0.333
epoch: 8      time: 8.76    loss: 183.210    train acc: 0.286    val acc: 0.331
epoch: 9      time: 8.73    lo

In [4]:
print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Augmenting test data...
Testing...
Testing Accuracy: 0.4266


### Testtttt

In [3]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 9
cwt_scale = 0.4

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 30
hidden_size = 50
lr = 2e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "cwt"
aug_2 = "window"


#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Data normalized...
Data shape: (2115, 22, 1000)
cwt applied...
fold 1
Initial fold shape: (1692, 198, 1000)
window applied...
Data shape: (16920, 198, 250)
RNN TYPE: GRU
WEIGHT DECAY: 0.001
LEARNING RATE: 0.002
Data prepared, model initialized, beginning training...
epoch: 1      time: 8.75    loss: 185.518    train acc: 0.255    val acc: 0.258
saving best model...
epoch: 2      time: 8.84    loss: 184.361    train acc: 0.263    val acc: 0.265
saving best model...
epoch: 3      time: 8.81    loss: 184.303    train acc: 0.263    val acc: 0.267
saving best model...
epoch: 4      time: 8.92    loss: 184.200    train acc: 0.271    val acc: 0.284
saving best model...
epoch: 5      time: 8.85    loss: 183.880    train acc: 0.274    val acc: 0.251
epoch: 6      time: 8.85    loss: 183.926    train acc: 0.275    val acc: 0.260
epoch: 7      time: 8.86    loss: 183.834    train acc: 0.273    val acc: 0.267
epoch: 8      time: 8.75    loss: 183.800    train acc: 0.275    val acc: 0.284
saving be

## Bandpass Filter Function

In [0]:
def bandpass_filter(X,low,high):
  N, C, T = X.shape
  out = np.zeros_like(X)
  nyq = 125 #nyquist frequency, highest able to be sensed for this data
  if high > nyq :
    high = nyq
  order = 9

  b, a = sig.butter(order, [low/nyq,high/nyq], btype='band')
  out = sig.lfilter(b, a, X, axis=-1)

  return out

## Tests with bandpass

### Test

In [5]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 200
window_stride = 50
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 9
cwt_scale = 1.0

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 30
hidden_size = 50
lr = 2e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "cwt"
aug_2 = "window"

#BANDPASS PARAMS
band_low = 7
band_high = 30

#bandpass the data
X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Data bandpass filtered...
Data normalized...
Data shape: (2115, 22, 1000)
cwt applied...
fold 1
Initial fold shape: (1692, 198, 1000)
window applied...
Data shape: (27072, 198, 200)
RNN TYPE: GRU
WEIGHT DECAY: 0.001
LEARNING RATE: 0.002
Data prepared, model initialized, beginning training...
epoch: 1      time: 10.77    loss: 294.517    train acc: 0.267    val acc: 0.298
saving best model...
epoch: 2      time: 10.29    loss: 293.576    train acc: 0.268    val acc: 0.274
epoch: 3      time: 10.23    loss: 293.399    train acc: 0.272    val acc: 0.312
saving best model...
epoch: 4      time: 10.30    loss: 292.906    train acc: 0.278    val acc: 0.253
epoch: 5      time: 10.26    loss: 292.644    train acc: 0.279    val acc: 0.345
saving best model...
epoch: 6      time: 10.30    loss: 291.994    train acc: 0.288    val acc: 0.326
epoch: 7      time: 10.26    loss: 290.250    train acc: 0.305    val acc: 0.359
saving best model...
epoch: 8      time: 10.21    loss: 288.586    train acc:

### Test Again

In [4]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 10
cwt_scale = 2

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 40
hidden_size = 50
lr = 2e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "cwt"
aug_2 = "window"

#BANDPASS PARAMS
band_low = 5
band_high = 35

#bandpass the data
X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Data bandpass filtered...
Data normalized...
Data shape: (2115, 22, 1000)
cwt applied...
fold 1
Initial fold shape: (1692, 220, 1000)
window applied...
Data shape: (16920, 220, 250)
RNN TYPE: GRU
WEIGHT DECAY: 0.001
LEARNING RATE: 0.002
Data prepared, model initialized, beginning training...
epoch: 1      time: 8.97    loss: 184.912    train acc: 0.261    val acc: 0.267
saving best model...
epoch: 2      time: 9.11    loss: 184.248    train acc: 0.271    val acc: 0.255
epoch: 3      time: 9.13    loss: 183.996    train acc: 0.274    val acc: 0.270
saving best model...
epoch: 4      time: 9.23    loss: 183.691    train acc: 0.279    val acc: 0.326
saving best model...
epoch: 5      time: 9.07    loss: 183.678    train acc: 0.279    val acc: 0.310
epoch: 6      time: 9.05    loss: 183.656    train acc: 0.279    val acc: 0.274
epoch: 7      time: 9.12    loss: 183.515    train acc: 0.286    val acc: 0.284
epoch: 8      time: 9.11    loss: 183.245    train acc: 0.287    val acc: 0.300
epoc

KeyboardInterrupt: ignored

### Using weird windowing

In [4]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 200
window_stride = 50
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 10
cwt_scale = 0.4

#RNN PARAMS
rnn_type = 'CNNGRUnet'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 2e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "cwt2"
aug_2 = "window"

#BANDPASS PARAMS
band_low = 7
band_high = 30

#bandpass the data
X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Data bandpass filtered...
Data normalized...
Data shape: (2115, 22, 1000)
cwt2 applied...
fold 1
Initial fold shape: (16920, 22, 1000)
window applied...
Data shape: (169200, 22, 200)
RNN TYPE: CNNGRUnet
WEIGHT DECAY: 0.001
LEARNING RATE: 0.002
Data prepared, model initialized, beginning training...
epoch: 1      time: 35.58    loss: 1810.645    train acc: 0.295    val acc: 0.360
saving best model...
epoch: 2      time: 35.24    loss: 1763.187    train acc: 0.344    val acc: 0.371
saving best model...
epoch: 3      time: 35.33    loss: 1740.999    train acc: 0.359    val acc: 0.366
epoch: 4      time: 35.42    loss: 1686.162    train acc: 0.400    val acc: 0.434
saving best model...
epoch: 5      time: 35.91    loss: 1654.259    train acc: 0.422    val acc: 0.434
saving best model...
epoch: 6      time: 35.33    loss: 1634.675    train acc: 0.432    val acc: 0.463
saving best model...
epoch: 7      time: 35.43    loss: 1622.712    train acc: 0.438    val acc: 0.406
epoch: 8      time: 3

### Repeating Josh's test

In [6]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 200
window_stride = 50
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.6

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 0
num_folds = 1

aug_1 = "cwt2"
aug_2 = "window"

#BANDPASS PARAMS
band_low = 7
band_high = 30

#bandpass the data
X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

Data bandpass filtered...
Data normalized...
Data shape: (2115, 22, 1000)
cwt2 applied...
fold 1
Initial fold shape: (13536, 22, 1000)
window applied...
Data shape: (216576, 22, 200)
RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.001
Data prepared, model initialized, beginning training...
epoch: 1      time: 155.23    loss: 2272.804    train acc: 0.330    val acc: 0.423
saving best model...
epoch: 2      time: 138.31    loss: 2054.214    train acc: 0.451    val acc: 0.467
saving best model...
epoch: 3      time: 134.66    loss: 1895.706    train acc: 0.512    val acc: 0.428
epoch: 4      time: 127.80    loss: 1770.024    train acc: 0.554    val acc: 0.457
epoch: 5      time: 138.26    loss: 1668.944    train acc: 0.587    val acc: 0.456
epoch: 6      time: 137.77    loss: 1587.056    train acc: 0.611    val acc: 0.474
saving best model...
epoch: 7      time: 135.07    loss: 1510.500    train acc: 0.635    val acc: 0.457
epoch: 8      time: 124.64    loss: 1452.823    train acc: 0.652  

### Larger Window

In [3]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 125
stft_stride = 5

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 0
num_folds = 1

aug_1 = "window"
aug_2 = "stft"

#BANDPASS PARAMS
band_low = 7
band_high = 30

#bandpass the data
X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
#X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_2,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale, vote_num=vote_num)

Data bandpass filtered...
Data normalized...
Data shape: (2115, 22, 1000)
window applied...
fold 1
Initial fold shape: (16920, 22, 250)
stft applied...
Data shape: (16920, 1672, 21)
RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.001
Data prepared, model initialized, beginning training...
epoch: 1      time: 5.82    loss: 182.745    train acc: 0.296    val acc: 0.293
saving best model...
epoch: 2      time: 5.47    loss: 179.293    train acc: 0.331    val acc: 0.317
saving best model...
epoch: 3      time: 5.51    loss: 175.721    train acc: 0.361    val acc: 0.327
saving best model...
epoch: 4      time: 5.50    loss: 171.396    train acc: 0.386    val acc: 0.330
saving best model...
epoch: 5      time: 5.52    loss: 168.784    train acc: 0.406    val acc: 0.328
epoch: 6      time: 5.54    loss: 163.903    train acc: 0.435    val acc: 0.333
saving best model...
epoch: 7      time: 5.48    loss: 159.917    train acc: 0.452    val acc: 0.337
saving best model...
epoch: 8      time: 5.53 

### STFT Grid Search

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_sizes = range(10,200,10)
stft_strides = range(1,10)

test_acc = np.empty((len(stft_sizes),len(stft_strides)))

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = None
aug_2 = "stft"

#BANDPASS PARAMS
#band_low = 7
#band_high = 30

#bandpass the data
#X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1)#,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)
for i in range(len(stft_sizes)):
  for j in range(len(stft_strides)):
    stft_size = stft_sizes[i]
    stft_stride = stft_strides[j]
    print('CURRENT STFT SIZE: %s CURRENT STFT STRIDE: %s'%(stft_size,stft_stride))
    for k in range(num_folds):
        # indicate hyperparameters here
        print ('fold {}'.format(k+1))
        X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
        X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
        print('Initial fold shape: {}'.format(X_train.shape))
        #apply second augmentation
        X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
        
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
        print('{} applied...'.format(aug_2))
        print('Data shape: {}'.format(X_train.shape))
        
        model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
        EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
        EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
        EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
        EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
        print('Data prepared, model initialized, beginning training...')
        best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

    print('Augmenting test data...')
    X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    #X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_2,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    print('Testing...')
    test_acc[i,j] = TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size,vote_num=vote_num,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    

Data bandpass filtered...
Data normalized...
Data shape: (2115, 22, 1000)
None applied...
CURRENT STFT SIZE: 10 CURRENT STFT STRIDE: 1
fold 1
Initial fold shape: (1692, 22, 1000)
stft applied...
Data shape: (1692, 132, 991)
RNN TYPE: GRU
WEIGHT DECAY: 0.001
LEARNING RATE: 0.001
Data prepared, model initialized, beginning training...
epoch: 1      time: 2.71    loss: 19.500    train acc: 0.240    val acc: 0.277
saving best model...
epoch: 2      time: 2.54    loss: 19.384    train acc: 0.274    val acc: 0.279
saving best model...
epoch: 3      time: 2.53    loss: 19.339    train acc: 0.264    val acc: 0.270
epoch: 4      time: 2.54    loss: 19.301    train acc: 0.290    val acc: 0.291
saving best model...
epoch: 5      time: 2.53    loss: 19.283    train acc: 0.285    val acc: 0.298
saving best model...
epoch: 6      time: 2.53    loss: 19.198    train acc: 0.292    val acc: 0.286
epoch: 7      time: 2.53    loss: 19.158    train acc: 0.291    val acc: 0.274
epoch: 8      time: 2.54    

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_sizes = range(40,200,10)
stft_strides = range(1,10)

test_acc = np.empty((len(stft_sizes),len(stft_strides)))

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = None
aug_2 = "stft"

#BANDPASS PARAMS
#band_low = 7
#band_high = 30

#bandpass the data
#X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1)#,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)
for i in range(len(stft_sizes)):
  for j in range(len(stft_strides)):
    stft_size = stft_sizes[i]
    stft_stride = stft_strides[j]
    print('CURRENT STFT SIZE: %s CURRENT STFT STRIDE: %s'%(stft_size,stft_stride))
    for k in range(num_folds):
        # indicate hyperparameters here
        print ('fold {}'.format(k+1))
        X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
        X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
        print('Initial fold shape: {}'.format(X_train.shape))
        #apply second augmentation
        X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
        
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
        print('{} applied...'.format(aug_2))
        print('Data shape: {}'.format(X_train.shape))
        
        model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
        EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
        EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
        EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
        EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
        print('Data prepared, model initialized, beginning training...')
        best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

    print('Augmenting test data...')
    X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    #X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_2,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    print('Testing...')
    test_acc[i,j] = TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size,vote_num=vote_num,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    

Data bandpass filtered...
Data normalized...
Data shape: (2115, 22, 1000)
None applied...
CURRENT STFT SIZE: 40 CURRENT STFT STRIDE: 1
fold 1
Initial fold shape: (1692, 22, 1000)
stft applied...
Data shape: (1692, 462, 961)
RNN TYPE: GRU
WEIGHT DECAY: 0.001
LEARNING RATE: 0.001
Data prepared, model initialized, beginning training...
epoch: 1      time: 5.64    loss: 19.565    train acc: 0.252    val acc: 0.255
saving best model...
epoch: 2      time: 5.43    loss: 19.292    train acc: 0.288    val acc: 0.270
saving best model...
epoch: 3      time: 5.42    loss: 19.129    train acc: 0.317    val acc: 0.232
epoch: 4      time: 5.51    loss: 19.029    train acc: 0.326    val acc: 0.253
epoch: 5      time: 5.41    loss: 18.852    train acc: 0.357    val acc: 0.260
epoch: 6      time: 5.44    loss: 18.719    train acc: 0.356    val acc: 0.229
epoch: 7      time: 5.44    loss: 18.592    train acc: 0.364    val acc: 0.217
epoch: 8      time: 5.41    loss: 18.374    train acc: 0.392    val ac

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_sizes = range(100,200,10)
stft_strides = range(4,10,2)

test_acc = np.empty((len(stft_sizes),len(stft_strides)))

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = None
aug_2 = "stft"

#BANDPASS PARAMS
#band_low = 7
#band_high = 30

#bandpass the data
#X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1)#,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)
for i in range(len(stft_sizes)):
  for j in range(len(stft_strides)):
    stft_size = stft_sizes[i]
    stft_stride = stft_strides[j]
    print('CURRENT STFT SIZE: %s CURRENT STFT STRIDE: %s'%(stft_size,stft_stride))
    for k in range(num_folds):
        # indicate hyperparameters here
        print ('fold {}'.format(k+1))
        X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
        X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
        print('Initial fold shape: {}'.format(X_train.shape))
        #apply second augmentation
        X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
        
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
        print('{} applied...'.format(aug_2))
        print('Data shape: {}'.format(X_train.shape))
        
        model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
        EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
        EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
        EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
        EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
        print('Data prepared, model initialized, beginning training...')
        best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

    print('Augmenting test data...')
    X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    #X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_2,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    print('Testing...')
    test_acc[i,j] = TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size,vote_num=vote_num,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    

Data bandpass filtered...
Data normalized...
Data shape: (2115, 22, 1000)
None applied...
CURRENT STFT SIZE: 100 CURRENT STFT STRIDE: 4
fold 1
Initial fold shape: (1692, 22, 1000)
stft applied...
Data shape: (1692, 1122, 226)
RNN TYPE: GRU
WEIGHT DECAY: 0.001
LEARNING RATE: 0.001
Data prepared, model initialized, beginning training...
epoch: 1      time: 2.79    loss: 19.681    train acc: 0.258    val acc: 0.243
saving best model...
epoch: 2      time: 2.56    loss: 19.082    train acc: 0.314    val acc: 0.236
epoch: 3      time: 2.56    loss: 18.801    train acc: 0.361    val acc: 0.253
saving best model...
epoch: 4      time: 2.60    loss: 18.487    train acc: 0.373    val acc: 0.265
saving best model...
epoch: 5      time: 2.58    loss: 18.068    train acc: 0.426    val acc: 0.243
epoch: 6      time: 2.58    loss: 17.475    train acc: 0.454    val acc: 0.258
epoch: 7      time: 2.57    loss: 17.016    train acc: 0.473    val acc: 0.248
epoch: 8      time: 2.57    loss: 16.324    tra

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_sizes = range(100,1000,50)
stft_strides = range(5,20,5)

test_acc = np.empty((len(stft_sizes),len(stft_strides)))

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = None
aug_2 = "stft"

#BANDPASS PARAMS
band_low = 7
band_high = 30

#bandpass the data
X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1)#,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)
for i in range(len(stft_sizes)):
  for j in range(len(stft_strides)):
    stft_size = stft_sizes[i]
    stft_stride = stft_strides[j]
    print('CURRENT STFT SIZE: %s CURRENT STFT STRIDE: %s'%(stft_size,stft_stride))
    for k in range(num_folds):
        # indicate hyperparameters here
        print ('fold {}'.format(k+1))
        X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
        X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
        print('Initial fold shape: {}'.format(X_train.shape))
        #apply second augmentation
        X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
        
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
        print('{} applied...'.format(aug_2))
        print('Data shape: {}'.format(X_train.shape))
        
        model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
        EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
        EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
        EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
        EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
        print('Data prepared, model initialized, beginning training...')
        best_model = TrainValRNN(verbose=False,model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

    print('Augmenting test data...')
    X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    #X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_2,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    print('Testing...')
    test_acc[i,j] = TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size,vote_num=vote_num,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    

Data bandpass filtered...
Data normalized...
Data shape: (2115, 22, 1000)
None applied...
CURRENT STFT SIZE: 100 CURRENT STFT STRIDE: 5
fold 1
Initial fold shape: (1692, 22, 1000)
stft applied...
Data shape: (1692, 1122, 181)
RNN TYPE: GRU
WEIGHT DECAY: 0.001
LEARNING RATE: 0.001
Data prepared, model initialized, beginning training...
saving best model...
saving best model...
saving best model...
saving best model...
Augmenting test data...
Testing...
Testing Accuracy: 0.2731
CURRENT STFT SIZE: 100 CURRENT STFT STRIDE: 10
fold 1
Initial fold shape: (1692, 22, 1000)
stft applied...
Data shape: (1692, 1122, 91)
RNN TYPE: GRU
WEIGHT DECAY: 0.001
LEARNING RATE: 0.001
Data prepared, model initialized, beginning training...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
Augmenting test data...
Testing...
Testing Accuracy: 0.2867
CURRENT STFT SIZE: 100 CURRENT STFT STRIDE: 15
fold 1
Initial fol

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_sizes = range(300,1000,100)
stft_strides = range(5,20,5)

test_acc = np.empty((len(stft_sizes),len(stft_strides)))

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = None
aug_2 = "stft"

#BANDPASS PARAMS
band_low = 7
band_high = 30

#bandpass the data
X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1)#,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)
for i in range(len(stft_sizes)):
  for j in range(len(stft_strides)):
    stft_size = stft_sizes[i]
    stft_stride = stft_strides[j]
    print('CURRENT STFT SIZE: %s CURRENT STFT STRIDE: %s'%(stft_size,stft_stride))
    for k in range(num_folds):
        # indicate hyperparameters here
        print ('fold {}'.format(k+1))
        X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
        X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
        print('Initial fold shape: {}'.format(X_train.shape))
        #apply second augmentation
        X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
        
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
        print('{} applied...'.format(aug_2))
        print('Data shape: {}'.format(X_train.shape))
        
        model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
        EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
        EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
        EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
        EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
        print('Data prepared, model initialized, beginning training...')
        best_model = TrainValRNN(verbose=False,model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

    print('Augmenting test data...')
    X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    #X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_2,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    print('Testing...')
    test_acc[i,j] = TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size,vote_num=vote_num,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    

Data bandpass filtered...
Data normalized...
Data shape: (2115, 22, 1000)
None applied...
CURRENT STFT SIZE: 300 CURRENT STFT STRIDE: 5
fold 1
Initial fold shape: (1692, 22, 1000)


## Bandpass + Window grid search on bandpass

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 200
window_stride = 20
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 125
stft_stride = 5

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = None
aug_2 = "window"

lows = range(2,16,2)
highs = range(20,50,5)


for low in lows:
  for high in highs:
    #BANDPASS PARAMS
    band_low = low
    band_high = high

    #bandpass the data
    X_train_valid_filt = bandpass_filter(X_train_valid, band_low, band_high)
    print('Data bandpass filtered with low = %s and high = %s ...'%(low,high))

    #make data N(0,1)
    mu = np.mean(X_train_valid_filt, axis=(0,2), keepdims = True) 
    std = np.std(X_train_valid_filt, axis=(0,2), keepdims = True)
    X_train_valid = standardize(X_train_valid_filt, mu, std)
    print('Data normalized...')
    print('Data shape: {}'.format(X_train_valid.shape))

    # apply first augmentation
    X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid_filt,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    print('{} applied...'.format(aug_1))
    train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


    for k in range(num_folds):
        # indicate hyperparameters here
        print ('fold {}'.format(k+1))
        X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
        X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
        print('Initial fold shape: {}'.format(X_train.shape))
        #apply second augmentation
        X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
        
        #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
        print('{} applied...'.format(aug_2))
        print('Data shape: {}'.format(X_train.shape))
        
        model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
        EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
        EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
        EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
        EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
        print('Data prepared, model initialized, beginning training...')
        best_model = TrainValRNN(verbose=False,model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

    print('Augmenting test data...')
    X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    #X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_2,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    print('Testing...')
    TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale, vote_num=vote_num)

Data bandpass filtered with high = 2 and low = 20 ...
Data normalized...
Data shape: (2115, 22, 1000)
None applied...
fold 1
Initial fold shape: (1692, 22, 1000)
window applied...
Data shape: (67680, 22, 200)
RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.001
Data prepared, model initialized, beginning training...
saving best model...
saving best model...
saving best model...
Augmenting test data...
Testing...
Testing Accuracy: 0.2889
Data bandpass filtered with high = 2 and low = 25 ...
Data normalized...
Data shape: (2115, 22, 1000)
None applied...
fold 1
Initial fold shape: (1692, 22, 1000)
window applied...
Data shape: (67680, 22, 200)
RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.001
Data prepared, model initialized, beginning training...
saving best model...
saving best model...
saving best model...
Augmenting test data...
Testing...
Testing Accuracy: 0.2144
Data bandpass filtered with high = 2 and low = 30 ...
Data normalized...
Data shape: (2115, 22, 1000)
None applied...
fold